In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('..')
from detector.anchor_generator import AnchorGenerator
from detector.training_target_creation import create_targets, match_boxes
from detector.input_pipeline import Pipeline

In [ ]:
def draw_boxes(image, boxes, color='red', fill=None):
    
    w, h = image.size
    scaler = np.array([h, w, h, w], dtype='float32')
    boxes = boxes.copy() * scaler
    
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    
    for box in boxes:
        ymin, xmin, ymax, xmax = box
        outline = color
        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
    return image_copy

# Create a graph

In [ ]:
tf.reset_default_graph()

params = {
    'min_dimension': 640,
    'batch_size': 14,
    'image_height': 640,
    'image_width': 640,
}
pipeline = Pipeline(
    ['/home/gpu1/datasets/COCO/coco_person/train_shards/shard-0000.tfrecords'],
    is_training=True, params=params
)
iterator = pipeline.dataset.make_one_shot_iterator()
features, labels = iterator.get_next()


image = features['images'][0]
image_width, image_height = tf.shape(image)[1], tf.shape(image)[0]
anchor_generator = AnchorGenerator(
    strides=[8, 16, 32, 64, 128],
    scales=[32, 64, 128, 256, 512],
    scale_multipliers=[1.0, 1.4142],
    aspect_ratios=[1.0, 2.0, 0.5]
)
anchors = anchor_generator(image_height, image_width)

# only the last level:
# n = anchor_generator.num_anchors_per_feature_map[-1]
# anchors = anchors[-n:]

groundtruth_boxes = labels['boxes'][0]
groundtruth_labels  = labels['labels'][0]

N = tf.shape(groundtruth_boxes)[0]
num_anchors = tf.shape(anchors)[0]
only_background = tf.fill([num_anchors], -1)

matches = tf.to_int32(tf.cond(
    tf.greater(N, 0),
    lambda: match_boxes(
        anchors, groundtruth_boxes,
        positives_threshold=0.5,
        negatives_threshold=0.4,
        force_match_groundtruth=True
    ),
    lambda: only_background
))
reg_targets, cls_targets = create_targets(
    anchors, groundtruth_boxes,
    groundtruth_labels, matches
)      

In [ ]:
with tf.Session() as sess:
    result = sess.run({
        'image': image, 
        'anchors': anchors,
        'matches': matches, 
        'boxes': groundtruth_boxes
    })
matched = np.where(result['matches'] >= 0)[0]
ignored = np.where(result['matches'] == -2)[0]

# Draw

In [ ]:
i = Image.fromarray((255.0*result['image']).astype('uint8'))
i = draw_boxes(i, result['anchors'][matched], color='blue', fill=(0, 0, 255, 5))
i = draw_boxes(i, result['boxes'], color='red', fill=(255, 0, 0, 50))
i = draw_boxes(i, result['anchors'][ignored], color='yellow')
i